## Notebook 4.6: *Do dataset categories influence relative similarity consistency?*


In [ ]:
import sys 
import pandas as pd 
import numpy as np
from pathlib import Path
from itertools import combinations
from scipy.stats import pearsonr, spearmanr
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.gridspec import GridSpec

from constants import ds_info_file, cat_name_mapping, fontsizes
from helper import load_ds_info, save_or_show
import textwrap

sys.path.append('..')
from scripts.helper import parse_datasets

In [ ]:
ds_list = parse_datasets('../scripts/webdatasets_w_insub10k.txt')
ds_list = list(map(lambda x: x.replace('/', '_'), ds_list))

ds_info = load_ds_info(ds_info_file)

## Storing information 
SAVE = True
corr_method = 'spearmanr' # spearmanr, pearsonr
storing_path = Path(f'/home/space/diverse_priors/results/plots/dist_r_coeff_dataset_cats_v2/{corr_method}')
if SAVE:
    storing_path.mkdir(parents=True, exist_ok=True)

## Load data
orig_sim_data = pd.read_csv('/home/space/diverse_priors/results/aggregated/model_sims/all_metric_ds_model_pair_similarity.csv')
orig_sim_data['Objective pair'] = orig_sim_data['Objective pair'].apply(eval)

combinations_objectives = [
                            ('all', 'all'), 
                            ('Self-Supervised', 'Self-Supervised'), 
                            ('Image-Text', 'Supervised'), 
                          ]

In [ ]:
def filter_data(mcat1, mcat2, sim_data, curr_suffix):
    if len(mcat1)>0 and len(mcat2)>0:
            sim_data_flat = sim_data[sim_data['Objective pair'].apply(lambda x: sorted(x) == sorted([mcat1, mcat2]))].copy()
            curr_suffix = f"_{cat_name_mapping[mcat1]}_{cat_name_mapping[mcat2]}" + curr_suffix
    else:
        sim_data_flat = sim_data
    return sim_data_flat, curr_suffix
    
    
def get_ds_combs(df):
    n_ds = df['DS'].nunique()
    available_ds = sorted(list(set(df['DS'].unique()).intersection(ds_list)))
    available_domains = sorted(ds_info['domain'].unique().tolist())
    # available_ds = sorted(available_ds, key = lambda x: available_domains.index(ds_info.loc[x, 'domain']))
    available_ds = sorted(available_ds, key = lambda x: (ds_info.loc[x, 'domain'], ds_info.loc[x, 'name']))
    combs_DS = list(combinations(available_ds, 2))
    corr_mat = pd.DataFrame(index=available_ds, columns=available_ds).astype('float')
    return available_ds, available_domains, combs_DS, corr_mat


def get_r_coeff(x,y, method='pearsonr'):
    if method=='pearsonr':
        corr, _  = pearsonr(x, y)
    elif method=='spearmanr':
        corr, _  = spearmanr(x, y)
    else:
        raise ValueError("Unknown method")
    return corr 


def get_two_ds_data(ds1, ds2, all_sims):
    data_2_ds = all_sims[all_sims['DS'].isin([ds1, ds2])].copy()
    data_2_ds['model_pair'] = data_2_ds['Model 1'] + ", " + data_2_ds['Model 2']
    ds_similarities = pd.pivot_table(
        data_2_ds,
        columns='DS',
        index='model_pair',
        values= 'Similarity value',
    )
    return ds_similarities 


def fill_corr_mat(df, combs_DS, corr_mat):
    for ds1, ds2 in combs_DS:
        ds_sims = get_two_ds_data(ds1, ds2, df)
        corr = get_r_coeff(ds_sims.values[:,0], ds_sims.values[:,1], method=corr_method)
        corr_mat.loc[ds1, ds2] = float(corr)
        corr_mat.loc[ds2, ds1] = float(corr)
    
    np.fill_diagonal(corr_mat.values, 1)
    return corr_mat


def rename_idx_cols(corr_mat, available_ds):
    new_naming = [ds_info.loc[ds, 'name'] for ds in available_ds]
    corr_mat.index = new_naming
    corr_mat.columns = new_naming
    return corr_mat


def get_all_correlations(df, corr_type, col_data):
    r_vals = []
    for ds1, ds2 in combinations(ds_list, 2):
        ds1_subset = df[df['DS'] == ds1]
        ds2_subset = df[df['DS'] == ds2]
        r_vals.append({
            'ds1': ds1,
            'ds2': ds2,
            'r coeff': get_r_coeff(ds1_subset[col_data].values, ds2_subset[col_data].values, corr_type)
            
        })

    return pd.DataFrame(r_vals)


def get_all_ds_corr_mat(sim_metric):
    all_sim_data = pd.read_csv(Path('/home/space/diverse_priors/results/aggregated/model_sims/all_metric_ds_model_pair_similarity.csv'))
    all_sim_data = all_sim_data[all_sim_data['DS'].isin(ds_list)]
    sim_data = all_sim_data[all_sim_data['Similarity metric'] == sim_metric]
    r_corrs = get_all_correlations(sim_data, corr_method, 'Similarity value')
    corr_mat = pd.DataFrame(columns=ds_list, index=ds_list, dtype=float)
    def add_entries(row):
        corr_mat.loc[row['ds1'],  row['ds2']] = row['r coeff']
        corr_mat.loc[row['ds2'],  row['ds1']] = row['r coeff']
    
    r_corrs.apply(add_entries , axis=1)
    np.fill_diagonal(corr_mat.values, 1)
    new_index = ds_info.loc[corr_mat.index, :].sort_values(['domain', 'name']).index 
    corr_mat = corr_mat.loc[new_index, new_index]
    corr_mat.index = ds_info.loc[corr_mat.index, 'name']
    corr_mat.columns = ds_info.loc[corr_mat.columns, 'name']
    return corr_mat, list(new_index), sorted(ds_info.loc[new_index, 'domain'].unique())


# def plot_heatmap(corr_mat, available_ds, available_domains, vmin = -0.2, vmax=1):
#     plt.figure(figsize=(9,7))

#     g = sns.heatmap(corr_mat, square=True, cmap='mako', cbar=False, vmin = vmin, vmax=vmax)
    
#     tmp = np.where(~(ds_info.loc[available_ds,'domain'].iloc[:-1].values == ds_info.loc[available_ds,'domain'].iloc[1:].values))[0]
#     tmp +=1
    
#     for val in tmp:
#         g.axhline(val, c='black', ls=":")
#         g.axvline(val, c='black', ls=":")
    
#     g.text(5 , -1.5, '\n'.join(available_domains[0].split(' ')), ha='center', va='top', fontsize=10, color='black')
#     g.text(13 , -1.5, '\n'.join(available_domains[1].split(' ')), ha='center', va='top', fontsize=10, color='black')
#     g.text(17.75 , -1, available_domains[2], ha='center', va='top', fontsize=10, color='black')
#     g.text(21.75 , -1, available_domains[3], ha='center', va='top', fontsize=10, color='black')
    
    
#     cbar = plt.colorbar(g.collections[0], ax=g, orientation='vertical', pad=0.025, aspect=40)
    
#     plt.tight_layout()
#     return plt.gcf()


def plot_heatmap_v2(ax, corr_mat, available_ds, available_domains, vmin = -0.44, vmax=1):

    sns.heatmap(corr_mat, square=True, cmap='mako', cbar=False, vmin = vmin, vmax=vmax, ax=ax)
    
    tmp = np.where(~(ds_info.loc[available_ds,'domain'].iloc[:-1].values == ds_info.loc[available_ds,'domain'].iloc[1:].values))[0]
    tmp +=1
    
    for val in tmp:
        ax.axhline(val, c='black', ls=":")
        ax.axvline(val, c='black', ls=":")
    
    ax.text(5 , -1.5, '\n'.join(available_domains[0].split(' ')), ha='center', va='top', fontsize=10, color='black')
    ax.text(13 , -1.5, '\n'.join(available_domains[1].split(' ')), ha='center', va='top', fontsize=10, color='black')
    ax.text(17.75 , -1, available_domains[2], ha='center', va='top', fontsize=10, color='black')
    ax.text(21.75 , -1, available_domains[3], ha='center', va='top', fontsize=10, color='black')
    
    ax.set_xlabel('')
    ax.set_ylabel('')
    return ax


def setup_figure(m, size_fig = 7, size_bar = 0.25, wspace=0.05):
    fig = plt.figure(figsize=(m*size_fig + size_bar, size_fig))
    gs = GridSpec(1, m+1, width_ratios=[size_fig]*m + [size_bar], wspace=wspace)

    # Create axes with shared x and y
    ax0 = fig.add_subplot(gs[0, 0])
    axs = [ax0] + [fig.add_subplot(gs[0, i], sharey=ax0, sharex=ax0) for i in range(1, m)]
    return fig, gs, axs


def add_colorbar(gs, cmap='mako', vmin=-0.44, vmax=1):
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))
    sm.set_array([])
    cax = fig.add_subplot(gs[0, -1])
    plt.colorbar(sm, cax=cax)


def update_suffix(suffix, mcat1, mcat2):
    def pp_str(x):
        return x.lower().replace(' ', '_')
    if mcat1 == mcat2:
        suffix += f"_2_{pp_str(mcat1)}"
    else:
        suffix += f"_{pp_str(mcat1)}_n_{pp_str(mcat2)}"
    return suffix

In [ ]:
m = len(combinations_objectives)

for sim_metric in ['CKA RBF 0.4', 'CKA linear']:
    print(sim_metric)
    
    sim_data = orig_sim_data[orig_sim_data['Similarity metric']==sim_metric].reset_index().copy()
    
    suffix = "_" + sim_metric.replace(" ", "_").lower()
    
    fig, gs, axs = setup_figure(m, size_fig = 6.5, size_bar = 0.25, wspace=0.05)

    for i, (mcat1, mcat2) in enumerate(combinations_objectives):
        ax = axs[i]
        if mcat1 == 'all' and mcat2 == 'all':
            corr_mat, available_ds, available_domains = get_all_ds_corr_mat(sim_metric)
        else:
            sim_data_flat, _ = filter_data(mcat1, mcat2, sim_data, suffix)
            available_ds, available_domains, combs_DS, corr_mat = get_ds_combs(sim_data_flat)
            corr_mat = fill_corr_mat(sim_data_flat, combs_DS, corr_mat)
            corr_mat = rename_idx_cols(corr_mat, available_ds)
        
        plot_heatmap_v2(ax, corr_mat, available_ds, available_domains)
        
        if i > 0:
            plt.setp(ax.get_yticklabels(), visible=False)
        
        suffix = update_suffix(suffix, mcat1, mcat2)

    add_colorbar(gs, cmap='mako', vmin=-0.44, vmax=1)

    save_or_show(plt.gcf(), storing_path / f'grouped_heatmap{suffix}.pdf', SAVE)

In [ ]:
# for sim_metric in ['CKA RBF 0.4', 'CKA linear']:
#     print(sim_metric)
    
#     sim_data = orig_sim_data[orig_sim_data['Similarity metric']==sim_metric].reset_index().copy()
    
#     suffix = "_" + sim_metric.replace(" ", "_").lower()
    
#     for mcat1, mcat2 in combinations_objectives:
        
#         sim_data_flat, curr_suffix = filter_data(mcat1, mcat2, sim_data, suffix)

#         available_ds, available_domains, combs_DS, corr_mat = get_ds_combs(sim_data_flat)

#         corr_mat = fill_corr_mat(sim_data_flat, combs_DS, corr_mat)

#         corr_mat = rename_idx_cols(corr_mat, available_ds)

#         fig = plot_heatmap(corr_mat, available_ds, available_domains, vmin = -0.2, vmax=1)

#         save_or_show(plt.gcf(), storing_path / f'grouped_heatmap{curr_suffix}.pdf', SAVE)

In [ ]:
# def get_domain_pair(row):
#     ds1 = row['DS 1']
#     ds2 = row['DS 2']
#     dm1 = ds_info[ds_info.name==ds1]['domain'].iloc[0]
#     dm2 = ds_info[ds_info.name==ds2]['domain'].iloc[0]
#     return ", ".join(sorted([dm1, dm2]))

# idxs = np.triu_indices_from(corr_mat.values, k=1)
# corr_mat.index[idxs[0]].values

# flattend_corr_mat = pd.DataFrame({
#     'DS 1': corr_mat.index[idxs[0]].values,
#     'DS 2': corr_mat.columns[idxs[1]].values,
#     'R coeff': corr_mat.values[idxs]
# })
# # flattend_corr_mat['R coeff'].describe()
# flattend_corr_mat['Domain pair'] = flattend_corr_mat.apply(get_domain_pair, axis=1)
# flattend_corr_mat.groupby('Domain pair')['R coeff'].describe().sort_values('mean')[['mean', 'std']]